### LangChain Rag exemple
Here we will use the help of mistral documentation to create our first
rag with langchain


In [47]:
from langchain_community.document_loaders import TextLoader
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
import requests
import os
api_key = os.environ["API_KEY"]



 ## Get Data essay and load it

In [39]:
response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

In [40]:
f = open('essay.txt', 'w')
f.write(text)
f.close()

In [41]:
loader = TextLoader("essay.txt")
docs = loader.load()

## Split into smaller chunks
Langchain enable us to use their prebuild TextSplitter into smaller chunk.

In [42]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 1000, chunk_overlap = 200
)
documents = text_splitter.split_documents(docs)

## Embed Chunks

In [43]:
embeddings = MistralAIEmbeddings(model="mistral-embed", api_key=api_key)

/data/Documents/projectsRag/projectRagLangchainChatBot/.venv/lib/python3.11/site-packages/langchain_mistralai/embeddings.py:180: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


## Create vector database to store embedded chunks

In [46]:
import time
from tenacity import retry, wait_exponential, stop_after_attempt

@retry(wait=wait_exponential(multiplier=1, min=4, max=60), stop=stop_after_attempt(5))
def create_vector_store(documents, embeddings):
	"""
	Creates a vector store using FAISS from the provided documents and embeddings.

	This function uses a retry mechanism to handle transient errors, with exponential backoff.

	Args:
		documents (list): A list of documents to be converted into vectors.
		embeddings (object): An embeddings object used to convert documents into vectors.

	Returns:
		FAISS: A FAISS vector store created from the provided documents and embeddings.

	Raises:
		Exception: If the function fails after the specified number of retry attempts.
	"""
	return FAISS.from_documents(documents, embeddings)

# Create the vector store with retry mechanism
vector = create_vector_store(documents, embeddings)
# Define a retriever interface
retriever = vector.as_retriever()

## Resume Steps below
1) ### Get data and load it
2) ### Split the data into smaller chunks
3) ### Verctorized our data

## Define our model using Mistral

In [52]:
model = ChatMistralAI(mistral_api_key=api_key, temperature=0.8)
# Define prompt template
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({
    "input": "What were the two main things the author worked on before college?"
})
response['answer']

'The two main things the author worked on before college were writing and programming. He wrote short stories and tried programming on an IBM 1401 using Fortran, but struggled to figure out what to do with it due to the limited input options available at the time.'